In [1]:
import yfinance as yf
import pandas as pd
import random
import numpy as np
import pendulum
from fitness_functions import *

In [2]:
class Volume:
    def __init__(self, value: float):
        self.value = value

class Dollar:
    def __init__(self, value: float):
        self.value = value


In [3]:
#Import the data.
btc_ts = yf.Ticker("BTC-USD").history(start='2016-01-01', end='2023-06-30')


In [4]:
base_columns_list = ['Open', 'High', 'Low', 'Close', 'Volume']
# btc_train = btc_train[base_columns_list]

# base_column_mapping = {}
# for i in base_columns_list:
#     base_column_mapping[i]=i.lower()
# btc_train = btc_train.rename(columns = base_column_mapping)
# btc_train.head(5)
# base_columns_list = [i.lower() for i in base_columns_list]

In [5]:
def create_ma_columns(df,column, lag):
    df[f"ma_{column.lower()}_{lag}"]= df[column].rolling(window = lag).mean()
    return df

def create_lagged_columns(df, column, lag):
    df[f"lag_{column.lower()}_{lag}"]= df[column].shift(lag)
    return df

def create_percentage_of_value(df, column, percentage:float = random.random()):
    df[f"percentage_{column.lower()}_{(round(percentage*100))}"]= df[column]*percentage
    return df

def create_difference(df, column1:str = "low", column2:str = "high"):
    df[f"difference_{column1.lower()}_{column2.lower()}"]= df[column2]-df[column1]
    return df

In [6]:
# Add columns to df:
for column in base_columns_list:
    for lag in range(2,50):
        create_ma_columns(df = btc_ts,column=column, lag = lag)
    for lag in range(1,50):
        create_lagged_columns(df = btc_ts,column=column, lag = lag)
    for lag in range(10):
        create_percentage_of_value(df = btc_ts,column=column)

C:\Users\khann\AppData\Local\Temp\ipykernel_18572\3535242035.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"lag_{column.lower()}_{lag}"]= df[column].shift(lag)
C:\Users\khann\AppData\Local\Temp\ipykernel_18572\3535242035.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"lag_{column.lower()}_{lag}"]= df[column].shift(lag)
C:\Users\khann\AppData\Local\Temp\ipykernel_18572\3535242035.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

In [7]:
btc_ts.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits,ma_open_2,ma_open_3,ma_open_4,...,lag_volume_41,lag_volume_42,lag_volume_43,lag_volume_44,lag_volume_45,lag_volume_46,lag_volume_47,lag_volume_48,lag_volume_49,percentage_volume_0
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00+00:00,430.721008,436.246002,427.515015,434.334015,36278900,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176394.368618
2016-01-02 00:00:00+00:00,434.622009,436.062012,431.869995,433.437988,30096600,0.0,0.0,432.671509,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146334.942750
2016-01-03 00:00:00+00:00,433.578003,433.743011,424.705994,430.010986,39633800,0.0,0.0,434.100006,432.973674,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192706.480266
2016-01-04 00:00:00+00:00,430.061005,434.516998,429.084015,433.091003,38477500,0.0,0.0,431.819504,432.753672,432.245506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,187084.347059
2016-01-05 00:00:00+00:00,433.069000,434.182007,429.675995,431.959991,34522600,0.0,0.0,431.565002,432.236003,432.832504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167854.930278


In [8]:
def pre_process(df):
    """Remove nulls and columns from the DF."""
    null_rows = df.isna().sum(axis=1)[df.isna().sum(axis=1)!=0]
    null_row_indices = list(null_rows.index)
    df.drop(null_row_indices,inplace=True)
    # sum(df.isna().sum(axis=1))
    df.drop(['Dividends','Stock Splits'], axis=1,inplace = True)
    return df

In [9]:
btc_ts = pre_process(btc_ts)

In [10]:
btc_ts.head()

,Open,High,Low,Close,Volume,ma_open_2,ma_open_3,ma_open_4,ma_open_5,ma_open_6,...,lag_volume_41,lag_volume_42,lag_volume_43,lag_volume_44,lag_volume_45,lag_volume_46,lag_volume_47,lag_volume_48,lag_volume_49,percentage_volume_0
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-19 00:00:00+00:00,422.730011,423.104004,417.604004,420.785004,55711300,419.651001,415.652669,412.097504,411.191602,408.148336,...,32278000.0,56993000.0,87562200.0,34042500.0,34522600.0,38477500.0,39633800.0,30096600.0,36278900.0,270878.102379
2016-02-20 00:00:00+00:00,421.601013,441.984009,421.601013,437.164001,93992096,422.165512,420.301005,417.139755,413.998206,412.926503,...,35995900.0,32278000.0,56993000.0,87562200.0,34042500.0,34522600.0,38477500.0,39633800.0,30096600.0,457006.040123
2016-02-21 00:00:00+00:00,437.773010,448.045990,429.076996,438.798004,89820704,429.687012,427.368011,424.669006,421.266406,417.960673,...,40450000.0,35995900.0,32278000.0,56993000.0,87562200.0,34042500.0,34522600.0,38477500.0,39633800.0,436724.001304
2016-02-22 00:00:00+00:00,438.989014,439.045013,432.916992,437.747986,85385200,438.381012,432.787679,430.273262,427.533008,424.220174,...,115607000.0,40450000.0,35995900.0,32278000.0,56993000.0,87562200.0,34042500.0,34522600.0,38477500.0,415157.803664
2016-02-23 00:00:00+00:00,438.255005,439.858002,417.821014,420.735992,85244896,438.622009,438.339010,434.154510,431.869611,429.320007,...,173888000.0,115607000.0,40450000.0,35995900.0,32278000.0,56993000.0,87562200.0,34042500.0,34522600.0,414475.621031


In [11]:
volume_columns = [col  for col in btc_ts.columns if "volume" in col.lower()]
dollar_columns = [col  for col in btc_ts.columns if "volume" not in col.lower()]

In [12]:
btc_train = btc_ts[:"2021-06-30"]
btc_test = btc_ts["2021-06-30":]

# Genetic Program

In [13]:
from deap import gp, creator, base, tools
import operator
from operator import or_, and_, gt
from fitness_functions import *

In [14]:
arg_names = list(btc_train.columns)
vol_args = [arg for arg in arg_names if "volume" in arg.lower()]
dol_args = [arg for arg in arg_names if "volume" not in arg.lower()]

In [15]:
n_args=len(arg_names)
pset = gp.PrimitiveSetTyped("main",[Volume]*len(vol_args) + [Dollar]*len(dol_args),bool)
#Rename the arguments:
arg_vol_mapping = {f"ARG{ind}": val for ind,val in enumerate(vol_args)}
pset.renameArguments(**arg_vol_mapping)
arg_dol_mapping = {f"ARG{len(vol_args)+ind}": val for ind,val in enumerate(dol_args)}
pset.renameArguments(**arg_dol_mapping)
#Check that all arguments were renamed:
unnamed_args=[i for i in pset.arguments if "ARG" in i]
if  unnamed_args:
    print(f"Some arguments were not renamed: {unnamed_args}")
# pset.addPrimitive(operator.lt, [float,float],bool)
pset.addPrimitive(gt, [Dollar,Dollar],bool)
pset.addPrimitive(lambda x:x ,[Dollar],Dollar, name="dollar placeholder")

pset.addPrimitive(gt, [Volume,Volume],bool)
pset.addPrimitive(lambda x:x ,[Volume],Volume, name="volume placeholder")


#Boolean operators:
pset.addPrimitive(and_, [bool,bool],bool)
pset.addPrimitive(or_,[bool,bool],bool)

# pset.addTerminal(True,bool)
# pset.addTerminal("ARG1",float)
# pset.addTerminal(False,bool)
for v_arg in vol_args:
        pset.addTerminal(v_arg,Volume)
for d_arg in dol_args:
        pset.addTerminal(d_arg,Dollar)

# --- Remove all the ARG terminals ---
pset.terminals[Volume] = [i for i in pset.terminals[Volume] if "ARG" not in i.name]
pset.terminals[Dollar] = [i for i in pset.terminals[Dollar] if "ARG" not in i.name]

In [16]:
[i for  i in pset.terminals]
[i.name for i in pset.terminals[Dollar] if 'ARG' in i.name]

[]

In [17]:
pset.arguments

['Volume',
 'ma_volume_2',
 'ma_volume_3',
 'ma_volume_4',
 'ma_volume_5',
 'ma_volume_6',
 'ma_volume_7',
 'ma_volume_8',
 'ma_volume_9',
 'ma_volume_10',
 'ma_volume_11',
 'ma_volume_12',
 'ma_volume_13',
 'ma_volume_14',
 'ma_volume_15',
 'ma_volume_16',
 'ma_volume_17',
 'ma_volume_18',
 'ma_volume_19',
 'ma_volume_20',
 'ma_volume_21',
 'ma_volume_22',
 'ma_volume_23',
 'ma_volume_24',
 'ma_volume_25',
 'ma_volume_26',
 'ma_volume_27',
 'ma_volume_28',
 'ma_volume_29',
 'ma_volume_30',
 'ma_volume_31',
 'ma_volume_32',
 'ma_volume_33',
 'ma_volume_34',
 'ma_volume_35',
 'ma_volume_36',
 'ma_volume_37',
 'ma_volume_38',
 'ma_volume_39',
 'ma_volume_40',
 'ma_volume_41',
 'ma_volume_42',
 'ma_volume_43',
 'ma_volume_44',
 'ma_volume_45',
 'ma_volume_46',
 'ma_volume_47',
 'ma_volume_48',
 'ma_volume_49',
 'lag_volume_1',
 'lag_volume_2',
 'lag_volume_3',
 'lag_volume_4',
 'lag_volume_5',
 'lag_volume_6',
 'lag_volume_7',
 'lag_volume_8',
 'lag_volume_9',
 'lag_volume_10',
 'lag_volu

In [18]:
# arg_vol_mapping = {f"ARG{ind}": val for ind,val in enumerate(vol_args)}
arg_vol_mapping = {f"ARG{len(vol_args)+ind}": val for ind,val in enumerate(dol_args)}
arg_vol_mapping

{'ARG99': 'Open',
 'ARG100': 'High',
 'ARG101': 'Low',
 'ARG102': 'Close',
 'ARG103': 'ma_open_2',
 'ARG104': 'ma_open_3',
 'ARG105': 'ma_open_4',
 'ARG106': 'ma_open_5',
 'ARG107': 'ma_open_6',
 'ARG108': 'ma_open_7',
 'ARG109': 'ma_open_8',
 'ARG110': 'ma_open_9',
 'ARG111': 'ma_open_10',
 'ARG112': 'ma_open_11',
 'ARG113': 'ma_open_12',
 'ARG114': 'ma_open_13',
 'ARG115': 'ma_open_14',
 'ARG116': 'ma_open_15',
 'ARG117': 'ma_open_16',
 'ARG118': 'ma_open_17',
 'ARG119': 'ma_open_18',
 'ARG120': 'ma_open_19',
 'ARG121': 'ma_open_20',
 'ARG122': 'ma_open_21',
 'ARG123': 'ma_open_22',
 'ARG124': 'ma_open_23',
 'ARG125': 'ma_open_24',
 'ARG126': 'ma_open_25',
 'ARG127': 'ma_open_26',
 'ARG128': 'ma_open_27',
 'ARG129': 'ma_open_28',
 'ARG130': 'ma_open_29',
 'ARG131': 'ma_open_30',
 'ARG132': 'ma_open_31',
 'ARG133': 'ma_open_32',
 'ARG134': 'ma_open_33',
 'ARG135': 'ma_open_34',
 'ARG136': 'ma_open_35',
 'ARG137': 'ma_open_36',
 'ARG138': 'ma_open_37',
 'ARG139': 'ma_open_38',
 'ARG140

In [19]:
arg_vol_mapping = {f"ARG{ind}": val for ind,val in enumerate(vol_args)}
pset.renameArguments(**arg_vol_mapping)

In [20]:
def plot_tree(expr, name:str = "tree"):
    nodes, edges, labels = gp.graph(expr)

    ### Graphviz Section ###
    import pygraphviz as pgv

    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(rf".\trees\{name}.png")

### Run the GP:

In [21]:
def generate(pset):
    run=True
    while run:
        try:
            expr = toolbox.individual()            
            #Remove all the Lambda functions:
            expr=  list(filter(lambda x: x.name!="dollar placeholder", expr))
            expr=  list(filter(lambda x: x.name!="volume placeholder", expr))
            if len(expr)>3:
                run=False
        except IndexError:
            continue
    # return gp.PrimitiveTree(expr)
    return creator.Individual(expr)

In [22]:
creator.create("fitness", base.Fitness, weights=(1,))
creator.create("Individual", gp.PrimitiveTree, fitness= creator.fitness)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("custom_individual",generate, pset)
toolbox.register("population", tools.initRepeat, list, toolbox.custom_individual)

expr1 = toolbox.custom_individual()
expr1

plot_tree(expr1)

In [23]:
ind = gp.PrimitiveTree(expr1)

In [24]:
pop = toolbox.population(n=10)

In [25]:
ind1=pop[0]
ind2=pop[2]

def cxOnePointCustom(ind1, ind2):
    size = min(len(ind1), len(ind2))
    retry = True
    while retry:
        try:
            cxpoint = random.randint(1, size - 1)
            internal_retry = True
            while internal_retry:
                print("Cross over point: ", cxpoint)
                if [type(i) for i in ind1[-cxpoint:]] == [type(i) for i in ind2[-cxpoint:]]:
                    if cxpoint==1 and ('volume' in ind1[-1].name.lower()) == ('volume' in ind2[-1].name.lower()):
                        ind1[-1:], ind2[-1:] = ind2[-1:], ind1[-1:]
                        return ind1, ind2
                    else:   
                        ind1[-cxpoint:], ind2[-cxpoint:] = ind2[-cxpoint:], ind1[-cxpoint:]
                        print(str(ind1[-cxpoint:]))
                        print(str(ind2[-cxpoint:]))
                        return ind1, ind2
                elif cxpoint>0:
                    cxpoint=cxpoint-1
                else:
                    internal_retry=False
            
        except ValueError:
            pass

c1,c2 = cxOnePointCustom(ind1, ind2)
print(ind1)
print(str(c1),'\n')
print(ind2)
print(str(c2))

Cross over point:  6
Cross over point:  7
Cross over point:  6
Cross over point:  3
[<deap.gp.Primitive object at 0x000001B3300A7880>, <deap.gp.Terminal object at 0x000001B3300CD4C0>, <deap.gp.Terminal object at 0x000001B3300CB940>]
[<deap.gp.Primitive object at 0x000001B3300A7880>, <deap.gp.Terminal object at 0x000001B3300C8180>, <deap.gp.Terminal object at 0x000001B3300CA200>]
or_(or_(or_(and_(gt('ma_open_46', 'ma_low_13'), gt('lag_close_17', 'lag_low_15')), gt('ma_open_2', 'lag_close_16')), or_(gt('lag_high_11', 'ma_high_8'), gt('ma_volume_7', 'ma_volume_3'))), gt('lag_close_13', 'lag_low_8'))
or_(or_(or_(and_(gt('ma_open_46', 'ma_low_13'), gt('lag_close_17', 'lag_low_15')), gt('ma_open_2', 'lag_close_16')), or_(gt('lag_high_11', 'ma_high_8'), gt('ma_volume_7', 'ma_volume_3'))), gt('lag_close_13', 'lag_low_8')) 

and_(gt('ma_volume_22', 'ma_volume_9'), and_(gt('lag_volume_21', 'lag_volume_48'), gt('ma_open_30', 'lag_high_13')))
and_(gt('ma_volume_22', 'ma_volume_9'), and_(gt('lag_vo

In [26]:
from deap.gp import Terminal

In [31]:
# [ for i in pop]
[ini.name for ini in pop[2] if isinstance(ini,Terminal)]

['ma_volume_22',
 'ma_volume_9',
 'lag_volume_21',
 'lag_volume_48',
 'ma_open_30',
 'lag_high_13']

In [ ]:
import fitness_functions

In [ ]:
[i for i in pset.terminals]

In [ ]:

terminal_dict = pset.terminals

print([i for i in pset.terminals[fitness_functions.Volume] if "ARG" not in i.name])
print(len(volume_columns))

In [ ]:
print([i.name for i in pset.terminals[fitness_functions.Volume] if "ARG" not in i.name])

In [ ]:
print([type(i) for i in ind1[-1:]])

In [ ]:
print('volume' in ind1[-1].name.lower(), 'volume' in ind2[-1].name)
print(('volume' in ind1[-1].name.lower()) == ('volume' in ind2[-1].name.lower()))

In [ ]:
ind = gp.PrimitiveTree(pop[0])
str(ind)

In [ ]:
function = gp.compile(expr=gp.PrimitiveTree(expr1),pset=pset)
function

#Get all the names of the arguments in the function, used in gp.py code:

## Method 1:
args_in_func=[]
for components in [i.name for i in expr]:
    if "ARG" in components:
        args_in_func.append(pset.arguments[int(components.replace("ARG",""))])
args_in_func = args_in_func + [a for a in [i.name for i in expr] if a in pset.arguments or a in pset.terminals]
args_in_func

##Method 2:
arg_in_func=[]
for arg in [ter for ter in [i.name for i in tree] if ter not in ["and_","or_","gt"]]+["volume"]:
    if "ARG" in arg:
        arg_in_func.append(pset.arguments[int(arg.replace("ARG",""))])
    else:
        arg_in_func.append(arg)

# Edit code to take values from DF:
code = str(tree).replace("'","")

for i in args_in_func:
    code = code.replace(i, f"df['{i}']")
code

## Create the Signal DF:

In [ ]:
signal_df  = pd.DataFrame(index=btc_train.index)

In [ ]:
# signal_df['signal'] = function(df = btc_train)

## Building Out trading strategy:

Assume purchases are done at the "Open" price:

In [ ]:
signal_df["Open"] = btc_train['Open']
signal_df.head()

## Calculate the MDD of a trade:

In [ ]:
# #Long code:
# change = []
# for ind, v in enumerate(ts_val):
#     if ind>0:
#         change.append(v-ts_val[ind-1])
# [v-ts_val[ind-1] for ind, v in enumerate(ts_val) if ind>0]
# mdd = min(change)
# mdd

In [ ]:
expr=pop[2]
expr1=pop[3]

ind=len(expr)-6
print(str(gp.PrimitiveTree(expr)))
print(str(gp.PrimitiveTree(expr[ind:])))
print([i.name for i in expr])
print([type(i) for i in expr])
print([i.arity for i in expr[ind:]])
plot_tree(expr,name="tree")
s=len(expr)

print("\n",'--'*30)

ind1=4
print(str(gp.PrimitiveTree(expr1)))
print(str(gp.PrimitiveTree(expr1[ind1:])))
print([i.name for i in expr1])
print([type(i) for i in expr1])
print([i.arity for i in expr[ind1:]])
plot_tree(expr1,name="tree1")

s1=len(expr1)

In [ ]:
cross = len(expr)-3
print([type(i) for i in expr[cross:]][:2])

In [ ]:
expr=pop[1]
expr1=pop[3]
sl = 1
cross = s-sl
cross1= s1-sl
print([i.name for i in expr[-sl:]])

print([i.name for i in expr1[-sl:]])


print([type(i) for i in expr[-sl:]] == [type(i) for i in expr1[-sl:]])
expr2 = expr.copy()
expr2[-sl:]=expr1[cross1:]
print(str(gp.PrimitiveTree(expr2)))

type(expr[2]) == type(expr1[3])
plot_tree(expr2, name="offspring")

In [ ]:
import deap
ind =5
print(expr[ind:] )
print([i for i in expr[ind:] if type(i)==deap.gp.Primitive])
print(str(gp.PrimitiveTree(expr[ind:])))
print('-'*40)
print(expr1[ind:] )
print([i for i in expr1[ind:] if type(i)==deap.gp.Primitive])
print(str(gp.PrimitiveTree(expr1[ind:])))

if not [i for i in expr[ind:] if type(i)==deap.gp.Primitive] and not [i for i in expr1[ind:] if type(i)==deap.gp.Primitive]:
    print(True)
else:
    print(False)


## Maximum theoretical profit:

In [ ]:

mtv = maximum_theoretical_value(df = btc_train)
mtv

In [ ]:
from deap import base, creator, tools, gp, algorithms

In [ ]:
toolbox.register("evaluate", fitness_function, df=btc_train, pset=pset)

In [ ]:
#Mutate entire branch:
def mutBranch(individual, pset, max_per_mutate=50):
    """Replaces a randomly chosen primitive from *individual* by a randomly
    chosen primitive with the same number of arguments from the :attr:`pset`
    attribute of the individual.

    :param individual: The normal or typed tree to be mutated.
    :returns: A tuple of one tree.
    """
    if len(individual) < 2:
        return individual,

    if (max_per_mutate*len(individual)/100) > 1 :
        "If the percentage of nodes that may be mutated is more than one randomly get the index."
        index=0
        while (len(individual) - index) > (max_per_mutate*len(individual)/100):
            "Ensure that maximum mutation portion is not exceeded."
            index = random.randrange(1, len(individual))
    else:
        index = len(individual)
    
    for i in range(index,len(individual)):
        node = individual[i]
        if node.arity == 0:  # Terminal
            term = random.choice(pset.terminals[node.ret])
            individual[i] = term
        else:  # Primitive
            prims = [p for p in pset.primitives[node.ret] if p.args == node.args]
            individual[i] = random.choice(prims)

    return individual,

def mutation_half(individual,mut_per, pset):
    if random.random()<0.5:
        return gp.mutNodeReplacement(individual, pset = pset)
    else:
        return mutBranch(individual, max_per_mutate = mut_per, pset=pset)

In [ ]:
toolbox.register("mate",       tools.cxOnePointCustom)
toolbox.register("select",     tools.selRoulette) 
toolbox.register("mutate",     mutation_half, pset=pset)

In [ ]:
hof   = tools.HallOfFame(maxsize=50)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0) 
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

# pop, logbook = algorithms.eaSimple(pop, toolbox,cxpb=0.7, mutpb=0.7, ngen=ngen, stats = stats, halloffame =hof)

In [ ]:
from operator import attrgetter

def GPAlgo(population, toolbox, cxpb, mutpb, ngen, elite_pop_size, stats=None,
             halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    store_generations = {}
    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        # assert (cxpb + mutpb) <= 1.0, (
        # "The sum of the crossover and mutation probabilities must be smaller "
        # "or equal to 1.0.")

        #Elitism:
        elite_pop = sorted(pop, key=attrgetter("fitness"), reverse=True)[:elite_pop_size]
        offspring = [toolbox.clone(ind) for ind in population]
        offspring[:elite_pop_size] = elite_pop

        # Apply crossover and mutation on the offspring
        for i in range(elite_pop_size+1, len(offspring)):
            if random.random() < cxpb:
                bi = random.randint(elite_pop_size+1, len(offspring)-1)
                offspring[bi], offspring[i] = toolbox.mate(offspring[bi],
                                                            offspring[i])
                del offspring[bi].fitness.values, offspring[i].fitness.values

        for i in range(elite_pop_size+1, len(offspring)):
            if random.random() < mutpb:
                mut_per = ((ngen+1-gen)/(ngen+1-gen))*100
                # print("Before Mut:",str(gp.PrimitiveTree(offspring[i])))
                offspring[i], = toolbox.mutate(offspring[i], mut_per=mut_per)
                # print("After Mut:",str(gp.PrimitiveTree(offspring[i])))
                del offspring[i].fitness.values

        elits_check = []
        for ind,val in enumerate(pop):
            elits_check.append(val == offspring[ind])
        # print(elits_check)
        print(sum(elits_check))
        # print(len([i for i in pop if i in offspring]))
        # print("Unique items in pop",set(pop), ' of ',len(pop))


        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        store_generations[f"gen{gen}"]={
            "pop":pop,
            "offspring":offspring
        }
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook, store_generations

In [ ]:
from operator import attrgetter
len(sorted(pop, key=attrgetter("fitness"), reverse=True)[:10])

In [ ]:
population, logbook, store_generations = GPAlgo(
    pop, 
    toolbox,
    cxpb=0.7, 
    mutpb=0.6, 
    ngen=2, 
    elite_pop_size= 10,
    stats = stats, 
    halloffame =hof
    )

In [ ]:
function = gp.compile(expr=gp.PrimitiveTree(pop[0]),pset=pset)
function(df = btc_ts)

In [ ]:
sum_same = []

for i in store_generations:
    same =[]
    for  k in range(len(store_generations[i]["pop"])):
        # print(store_generations[i]["pop"][k])
        # print(store_generations[i]["offspring"][k])
        is_same = (store_generations[i]["offspring"][k]==store_generations[i]["pop"] )
        same.append(is_same)
    pop_in_off = [ind in store_generations[i]["offspring"] for ind in store_generations[i]["pop"]]
    print('',pop_in_off)
    print(sum(pop_in_off))
    # print(same) 
    print(sum(same))
    print("----"*10)
    sum_same.append(sum(same))
print(sum_same)

In [ ]:
import matplotlib.pyplot as plt
y = [i['max'] for i in logbook]
plt.plot(range(len(y)), y)

In [ ]:
best_solution = hof.items[0]
tc = 0.01
print("TRAIN")
print("Buy and hold:",(1000/btc_train.iloc[0]['Open'])*btc_train.iloc[-1]['Open']*(1-tc)**2)
print("mtv         :",maximum_theoretical_value(btc_train))
print("strat value :",trading_strat(individual = best_solution, df=btc_train,pset=pset)[0],'\n')

print("TEST")
print("Buy and hold:",(1000/btc_test.iloc[0]['Open'])*btc_test.iloc[-1]['Open']*(1-tc)**2)
print("mtv         :",maximum_theoretical_value(btc_test))
print("strat value :",trading_strat(individual = best_solution, df=btc_test,pset=pset)[0])

In [ ]:
expr=pop[2]
expr1=pop[3]

ind=len(expr)-6
print(str(gp.PrimitiveTree(expr)))
print(str(gp.PrimitiveTree(expr[ind:])))
print([i.name for i in expr])
print([type(i) for i in expr])
print([i.arity for i in expr[ind:]])
plot_tree(expr,name="tree")
s=len(expr)

print("\n",'--'*30)

ind1=4
print(str(gp.PrimitiveTree(expr1)))
print(str(gp.PrimitiveTree(expr1[ind1:])))
print([i.name for i in expr1])
print([type(i) for i in expr1])
print([i.arity for i in expr[ind1:]])
plot_tree(expr1,name="tree1")

s1=len(expr1)

In [ ]:
#Check best solution:
print(str(best_solution))
plot_tree(expr = best_solution,
          name=f"tree_best_solution_{pendulum.now().format("MM-DD_HH-mm-ss")}")
function = gp.compile(expr=gp.PrimitiveTree(best_solution),pset=pset)
signal_df  = pd.DataFrame(index=btc_ts.index)
signal_df['Signal'] = function(df = btc_ts)
signal_df["Open"] = btc_ts['Open']
plt.figure(figsize=(15,7))
# btc_ts['Open'].plot()
x_btc = list(btc_ts.index)
y_btc = list(btc_ts['Open'])

x_sig = list(signal_df[signal_df['Signal']].index)
y_sig= list(signal_df[signal_df['Signal']]['Open'])
plt.plot(x_btc,y_btc,'k')
plt.plot(x_sig,y_sig,'y')

plt.figure(figsize=(15,7))
plt.plot(x_btc[-500:],y_btc[-500:],'k')
plt.plot(x_sig[-500:],y_sig[-500:],'y')

In [ ]:
# Get random Profit
def random_trading(df, val = 1000):
    ts_val = [val]
    ts_df = pd.DataFrame(columns = ["value"])
    long=False

    for cnt,row in enumerate(df['Open'].to_frame().iterrows()):
        gd = random.random()
        if  gd<1/3:
            "Try to buy for 33% of the time"
            if long:
                pass
            else:
                shares = ((1-tc)*val)/row[1]['Open']
                long=True
        elif gd <2/3:
            if long:
                val = (1-tc)*shares*row[1]['Open']
                long=False
            else:
                pass       
        else:
            pass
            
            long=False
        if cnt == len(df) and long:
            val = shares*row[1]['Open']
        ts_val.append(val)
        ts_df.loc[row[0]] = val
    return val, ts_val, ts_df


In [ ]:
val_r, ts_val_r, ts_df_r = random_trading(btc_ts)
print("Random trading strategy value: ",val_r)
ts_df_r.plot()

In [ ]:
ts_df.plot()
# ts_df["2021-06-30":].plot()

In [ ]:
plt.figure()
btc_train["Open"].plot(figsize=(17,7))

In [ ]:
btc_train_open = btc_train['Open']
x = btc_train_open.reset_index()['Open']

# Get peaks and troughs
peaks, _ = find_peaks(x)
troughs, _ = find_peaks(-x)

peaks_troughs = list(peaks)+list(troughs)
#Add the enedpoints to the local maximum and minimums.
if 0 not in peaks_troughs:
    peaks_troughs.append(0)
if len(btc_train_open)-1 not in peaks_troughs:
    peaks_troughs.append(len(btc_train_open)-1)

peaks_troughs = sorted(peaks_troughs)
peaks_troughs

#Work out maximum theoretical value using an initial investement of $1000 by default.
profit=0
val = 1000
no_tc_val = 1000
for ind,price in enumerate(btc_train_open.iloc[peaks_troughs]):
    if ind>1 and price > btc_train_open.iloc[peaks_troughs].iloc[ind-1]:
        # Calculate 
        pot_val= (val*price/btc_train_open.iloc[peaks_troughs].iloc[ind-1])*(1-tc)**2
        if pot_val>val:
            val=pot_val        
# no_tc
if ind>1 and price > btc_train_open.iloc[peaks_troughs].iloc[ind-1]:
    no_tc_val = (no_tc_val*price/btc_train_open.iloc[peaks_troughs].iloc[ind-1])

val

In [ ]:
eth_ts = yf.Ticker("ETH-USD").history(start='2016-01-01', end='2023-06-30')

base_columns_list = ['Open', 'High', 'Low', 'Close', 'Volume']

for column in base_columns_list:
    for lag in range(2,50):
        create_ma_columns(df = eth_ts,column=column, lag = lag)
    for lag in range(1,50):
        create_lagged_columns(df = eth_ts,column=column, lag = lag)
    for lag in range(10):
        create_percentage_of_value(df = eth_ts,column=column)

print("TEST ETH")
print("Buy and hold:",(1000/eth_ts.iloc[0]['Open'])*eth_ts.iloc[-1]['Open']*(1-tc)**2)
print("mtv         :",maximum_theoretical_value(eth_ts))
print("strat value :",trading_strat(individual = best_solution, df=eth_ts, pset=pset)[0])

In [ ]:
eth_ts['Open'].plot()

In [ ]:
deap.tools.support.HallOfFame

In [ ]:
logbook